In [1]:
import requests

auth_url = "https://signin.tradestation.com/authorize"

public_key = "r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4"
private_key = "hFBk8xgV_UGJEUFnxVW-AFz6YToqZwdvM-48x5wLUQhzKiR99r2w780hL0giBfvd"

params = {"response_type": "code", "client_id": public_key, "redirect_uri": "http://localhost", "audience": "https://api.tradestation.com", "state": "STATE", 
        "scope": "openid offline_access profile MarketData ReadAccount Trade Crypto Matrix OptionSpreads"}

response = requests.get(auth_url, params=params)
response.history
for resp in response.history:
        print(resp.status_code, resp.url)

302 https://signin.tradestation.com/authorize?response_type=code&client_id=r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4&redirect_uri=http%3A%2F%2Flocalhost&audience=https%3A%2F%2Fapi.tradestation.com&state=STATE&scope=openid+offline_access+profile+MarketData+ReadAccount+Trade+Crypto+Matrix+OptionSpreads


In [ ]:
import requests

token_url = "https://signin.tradestation.com/oauth/token"

authorization_code = "xxMZvFaqtX95pCIj"

headers = {"content-type": "application/x-www-form-urlencoded"}
data = {"grant_type": "authorization_code", "client_id": public_key, "client_secret": private_key, "code": authorization_code, "redirect_uri": "http://localhost"}

r = requests.post(token_url, headers=headers, data=data).json()
print(r)

access_token = r["access_token"]
refresh_token = r["refresh_token"]
id_token = r["id_token"]

In [2]:
import requests
refresh_token = "lzijZJe0VUnjFEzq6dqWK-Q3rqhzyoq7kkrDkqVoJvoYn"
def refresh():
    token_url = "https://signin.tradestation.com/oauth/token"

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "refresh_token", "client_id": public_key, "client_secret": private_key, "refresh_token": refresh_token}

    r = requests.post(token_url, headers=headers, data=data).json()

    access_token = r["access_token"]
    return access_token

In [ ]:
import requests
import pandas as pd
import json
import csv

url = "https://api.tradestation.com/v3/marketdata/barcharts/TSLA"

headers = {"Authorization": f"Bearer {access_token}"}

params = {"unit": "Minute", "firstdate": "2016-12-05T21:00:00Z", "lastdate": "2016-12-06T21:00:00Z"}

response = requests.request("GET", url, headers=headers, stream=True, params=params)
print(response)
if response.status_code == 401:
    access_token = refresh()
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.request("GET", url, headers=headers, stream=True, params=params)
    print(response)
response = response.json()
    
fieldnames = list(response["Bars"][0].keys())
with open("tradestation_data/tsla_2016_12_05_2016_12_06.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in response["Bars"]:
        writer.writerow(row)

In [ ]:
def stream_lvl1_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/quotes/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue   

            for line in response.iter_lines():
                if line:
                    row = json.loads(line)
                    if "Error" in row:
                        print(f"{symbol} {response} {response.text}")
                        continue
                    if "Heartbeat" in row:
                        continue
                    print(row)
        except Exception as e:
            print(f"{symbol} {e}")
            continue

In [ ]:
import os
import csv
import json
import pandas as pd
import datetime
import time

def stream_lvl2_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/marketdepth/aggregates/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue            
            
            for line in response.iter_lines():
                if line:
                    bids_and_asks = json.loads(line)
                    if "Heartbeat" in bids_and_asks:
                        continue
                    print(bids_and_asks)
        except Exception as e:
            print(f"{symbol} {e}")
            break
                    
symbols = "TSLA"
stream_lvl2_quotes(symbols)

In [5]:
import requests
account_id = "11655345"
def buy_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
buy_share("TSLA")

{"Orders":[{"Message":"Sent order: Buy 1 TSLA @ Market","OrderID":"967793406"}]}



In [11]:
import requests
def sell_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "SELL",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
sell_share("TSLA")

{"Orders":[{"Message":"Sent order: Sell 1 TSLA @ Market","OrderID":"967802392"}]}



In [13]:
import requests
def check_status():
    access_token = refresh()
    url = "https://api.tradestation.com/v3/brokerage/accounts/11655345/orders"

    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request("GET", url, headers=headers)

    print(response.text)
check_status()

{"Orders":[{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:10Z","Currency":"USD","Duration":"DAY","FilledPrice":"0","Legs":[{"QuantityOrdered":"1","ExecQuantity":"0","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK"}],"OrderID":"967805697","OpenedDateTime":"2023-06-07T19:53:10Z","OrderType":"Market","PriceUsedForBuyingPower":"224.98","RejectReason":"EC501: Day trading margin rules, this trade can possibly designate you as a pattern day trader, please call the Trade Desk at 800-871-3563 for more information.","Routing":"Intelligent","Status":"REJ","StatusDescription":"Rejected","UnbundledRouteFee":"0"},{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:00Z","Currency":"USD","Duration":"DAY","FilledPrice":"224.9699","Legs":[{"OpenOrClose":"Open","QuantityOrdered":"1","ExecQuantity":"1","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK","ExecutionPrice":"224.9699"}],"OrderID":

In [44]:
import datetime
import requests
account_id = "11655345"
def store_price(symbol):
    url = "https://api.tradestation.com/v3/orderexecution/orderconfirm"
    access_token = refresh()
    payload = {
        "AccountID": account_id,
        "Symbol": symbol,
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {"content-type": "application/json", "Authorization": f"Bearer {access_token}"}

    order_time = datetime.datetime.utcnow()
    time = order_time.replace(second=order_time.second - order_time.second%10).strftime("%Y-%m-%dT%H:%M:%SZ")
    response = requests.request("POST", url, json=payload, headers=headers).json()
    print(response)
    print(response["Confirmations"][0])
    buy_price = response["Confirmations"][0]["EstimatedPrice"]
    print(buy_price)
store_price("TSLA")

{'Confirmations': [{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}]}
{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}
226.53


In [ ]:
import requests

access_token = refresh()
account_id = "11655345"
url = f"https://api.tradestation.com/v3/brokerage/stream/accounts/{account_id}/orders"

headers = {"Authorization": f"Bearer {access_token}"}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)

In [2]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()

In [4]:
import numpy as np
import pandas as pd
def calc_real_gain(symbol, date, hour, price_dif_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    start = f'{date}T{hour}:00:00Z'
    end = f'{date}T{hour+1}:00:00Z'
    query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
    df = df.loc[df["Dif"] >= price_dif_threshold]
    
    df["VolumeSum"] = df["TotalSize"] * df["Dif"]
    df = df.rename(columns={"CurTime": "Time"})
    sum_df = pd.DataFrame(df.groupby("Time")["VolumeSum"].sum()).reset_index()
    sum_df = sum_df[(sum_df["VolumeSum"] >= size_threshold) | (sum_df["VolumeSum"] <= -size_threshold)]
    sum_df["Long"] = sum_df["VolumeSum"] >= size_threshold
    sum_df["Short"] = sum_df["VolumeSum"] <= -size_threshold

    query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
    final_df = pd.merge(sum_df, df_3, on="Time")
    final_df = final_df[["Time", "VolumeSum", "Long", "Short", "BUY", "SELL"]]
    final_df = final_df[final_df["BUY"].notna() & final_df["SELL"].notna()]

    if len(final_df) < 2:
        return 0

    final_df["Buy Action"] = final_df["Long"] & final_df["Short"].shift()
    if final_df.iloc[0, final_df.columns.get_loc("Long")]:
        final_df.iloc[0, final_df.columns.get_loc("Buy Action")] = True
    final_df.iloc[len(final_df) - 1, final_df.columns.get_loc("Buy Action")] = False
    final_df["Sell Action"] = final_df["Short"] & final_df["Long"].shift()
    if final_df.iloc[len(final_df) - 2, final_df.columns.get_loc("Long")]:
        final_df.iloc[len(final_df) - 1, final_df.columns.get_loc("Sell Action")] = True
    #final_df.to_csv("tradestation_data/TSLA_test.csv")

    final_df.loc[final_df["Buy Action"], "Gain"] = -pd.to_numeric(final_df.loc[final_df["Buy Action"], "BUY"]) 
    final_df.loc[final_df["Sell Action"], "Gain"] = pd.to_numeric(final_df.loc[final_df["Sell Action"], "SELL"])
    #print(final_df)
    ret = final_df["Gain"].sum()
    # for i in range(1, 24, 1):
    #     final_df[f"{i}0sec Dif"] = final_df["SELL"].shift(-i) - final_df["BUY"]
    #     final_df[f"{i}0sec Short Dif"] = final_df["SELL"] - final_df["BUY"].shift(-i)
    #     final_df.loc[final_df["Long"], f"{i}0sec Gain"] = final_df[f"{i}0sec Dif"]
    #     final_df.loc[final_df["Short"], f"{i}0sec Short Gain"] = final_df[f"{i}0sec Short Dif"]
    #     ret_list.append([final_df[f"{i}0sec Gain"].sum(), final_df[f"{i}0sec Short Gain"].sum()])
    # final_df = final_df[final_df["VolumeSum"].notnull()]
    # final_df.to_csv(f"tradestation_data/{symbol}_{date}_real_gain.csv")
    return ret
symbol = "TSLA"
#price_dif_threshold = .03
size_threshold = 1
# for size_threshold in range(1, 100, 1):
days = ["08", "09", "12", "13", "14", "15"]
days = ["13", "14", "15"]
hour = 13
#for hour in range(14, 20):
log_list = []
i = 100
for price_dif_threshold in np.arange(.2, -.1, -.001):
    entry = [price_dif_threshold, size_threshold]
    total = 0
    for day in days:
        date = f"2023-06-{day}"
        entry.append(calc_real_gain(symbol, date, hour, price_dif_threshold, size_threshold))
        total += entry[-1]
    entry.append(total)
    log_list.append(entry)
    if i == 100:
        print(log_list[-1])
        i = 0
    i += 1
out_df = pd.DataFrame(log_list)
out_df.to_csv(f"tradestation_data/TSLA_{hour}_threshold_tests.csv")


[0.2, 1, 0.2599999999999909, 0, -0.39999999999997726, -0.13999999999998636]
[0.09999999999999992, 1, 0.2599999999999909, -0.8499999999999659, 3.7600000000000193, 3.1700000000000443]
[-1.6653345369377348e-16, 1, -2.6900000000001683, -5.110000000000184, 3.920000000000158, -3.8800000000001944]
[-0.10000000000000026, 1, -1.7899999999999636, -1.7400000000004638, -1.6899999999999409, -5.220000000000368]


In [14]:
def get_max_volume(symbol, start_date, end_date):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    return df["TotalSize"].max(), df["TotalSize"].min(), df["TotalSize"].mean(), df["Dif"].max(), df["Dif"].min(), df["Dif"].mean() 

symbol = "TSLA"
start_date = "08"
end_date = "14"
print(get_max_volume(symbol, start_date, end_date))

(225600.0, 1.0, 487.7769264200571, 5.219999999999999, -250.45, -3.205123899786772)


In [8]:
def get_spread(symbol, start_date, end_date):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    start = f'2023-06-{start_date}T13:30:00Z'
    end = f'2023-06-{end_date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    query = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query)))
    df_3 = df_3.rename(columns={"Time": "CurTime"})
    df = pd.merge(df_1, df_2, on="CurTime")
    print(df)
    df = pd.merge(df, df_3, on="CurTime")
    print(df)
    df = df[df["BUY"].notnull() & df["SELL"].notnull() & df["Last"].notnull()]
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df = df.loc[df["Dif"] >= 0]
    df["BUY"] = pd.to_numeric(df["BUY"])
    df["SELL"] = pd.to_numeric(df["SELL"])
    df["Buy Dif"] = df["BUY"] - df["Last"]
    df["Sell Dif"] = df["Last"] - df["SELL"]
    df["Spread"] = df["BUY"] - df["SELL"]
    print(df["BUY"].min())
    print(df["Last"].min())
    return df["Buy Dif"].max(), df["Buy Dif"].min(), df["Buy Dif"].mean(), df["Sell Dif"].max(), df["Sell Dif"].min(), df["Sell Dif"].mean(), df["Spread"].max(), df["Spread"].min(), df["Spread"].mean()

symbol = "TSLA"
start_date = "13"
end_date = "14"
print(get_spread(symbol, start_date, end_date))

                           _id_x               CurTime             GetTime_x  \
0       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
1       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
2       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
3       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
4       64886f587199f685ff39196a  2023-06-13T13:30:00Z  2023-06-13T13:29:59Z   
...                          ...                   ...                   ...   
151611  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151612  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151613  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151614  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   
151615  648a1c367199f685ff4cb001  2023-06-14T19:59:50Z  2023-06-14T19:59:49Z   

       Symbol             TradeTime    